In [1]:
import csv
import json
import os
import requests

In [4]:
def fetch_manifest_data(manifest_url):
    response = requests.get(manifest_url)
    response.raise_for_status()
    return response.json()

def create_collection_manifest(manifest_urls, output_path):
    collection_manifest = {
        "@context": "http://iiif.io/api/presentation/2/context.json",
        "@id": "http://example.org/iiif/collection.json",
        "@type": "sc:Collection",
        "label": "Collection of IIIF Manifests",
        "manifests": []
    }

    for url in manifest_urls:
        manifest_data = fetch_manifest_data(url)
        manifest_id = manifest_data.get("@id", manifest_data.get("id"))
        if manifest_id:
            collection_manifest["manifests"].append({
                "@id": manifest_id,
                "@type": "sc:Manifest",
                "label": manifest_data.get("label", "No label")
            })
        else:
            print(f"Warning: '@id' and 'id' not found in manifest data from {url}")

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w') as f:
        json.dump(collection_manifest, f, indent=2)

def main(input_csv):
    output_json = './output/collection.json'

    manifest_urls = []
    with open(input_csv, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            manifest_urls.append(row['manifest_json'])

    create_collection_manifest(manifest_urls, output_json)

if __name__ == "__main__":
    input_csv = '/Users/maples/GitHub/Paper2Pixels/data/collection.csv'
    main(input_csv)